In [1]:
grid = get(load("../1.Upscaling/upscaling_ba/ba_grid_5.Rdata"))

In [4]:
source("Funciones_FS.r")


    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.1.3 (2021-08-04) is loaded


Get the latest stable version (2.1.5) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Warning message:
“no DISPLAY variable so Tk is not available”
visualizeR version 1.6.1 (2021-03-11) is loaded


Get the latest stable version (1.6.3) using <devtools::install_github('SantanderMetGroup/visualizeR')>

Please see 'citation("visualizeR")' to cite this package.



In [214]:
func.applyDataFrame <- function (row, func = mean){
    subgrid <- subsetGrid(grid, season = row["Mes"], lonLim = row["X"], latLim  = row["Y"])
    serie <- func(subgrid$Data)
    return (serie)
}

In [ ]:
parallel_apply <- function(i) {
  inicio <- filas_df_params_mes_inicios[i]
  final <- filas_df_params_mes_finales[i]
  parametros_mes <- df_params[inicio:final, ]
  resultado_mes <- apply(parametros_mes, 1, func.applyDataFrame)
  return(resultado_mes)
}

In [239]:
func.toDataFrame <- function(grid, func = mean){
    lon = grid$xyCoords$x
    lat = grid$xyCoords$y
    df_params <- expand.grid( "X" = lon, "Y" = lat, "Mes" = c(1:12))
    df.seriesTemporales <-  expand.grid( "X" = lon, "Y" = lat)
    vector_mes <- rep(0, nrow(df.seriesTemporales))
    df.seriesTemporales <- cbind(df.seriesTemporales, '1'=vector_mes, '2'=vector_mes, '3'=vector_mes, '4'=vector_mes, '5'=vector_mes, '6'=vector_mes, '7'=vector_mes, '8'=vector_mes,'9'=vector_mes,'10'=vector_mes,'11'=vector_mes, '12'=vector_mes)
    filas_df_params_mes <- (seq(0, nrow(df_params), length(lon) * length(lat)))
    filas_df_params_mes <- (seq(0, nrow(df_params), length(lon) * length(lat)))
    filas_df_params_mes_finales = c()
    for (i in filas_df_params_mes[2:length(filas_df_params_mes)]){
        filas_df_params_mes_finales = c(filas_df_params_mes_finales, i)
    }

    filas_df_params_mes_inicios = c(1)
    for (i in seq(1,12)){
            lista_finales = filas_df_params_mes_finales[2:length(filas_df_params_mes_finales)-1]
            filas_df_params_mes_inicios = c(filas_df_params_mes_inicios, lista_finales[i]+1)    
    }
    filas_df_params_mes_inicios = na.omit(filas_df_params_mes_inicios)
    progreso_total <- 12
    progreso <- 0
    progreso_pct <- (progreso / progreso_total) * 100
    cat(sprintf("\r%.2f%% completado", progreso_pct))
    for (i in seq(1,12)){
        inicio <- filas_df_params_mes_inicios[i]
        final <- filas_df_params_mes_finales[i]
        parametros_mes <- df_params[inicio:final,]
        resultado_mes <- apply(parametros_mes, 1, func.applyDataFrame)
        df.seriesTemporales[[as.character(i)]] <- resultado_mes
        progreso <- progreso + 1
        progreso_pct <- (progreso / progreso_total) * 100
        cat(sprintf("\r%.2f%% completado", progreso_pct))
        flush.console()
    }
    return(df.seriesTemporales)
    
}

In [240]:
a = Sys.time()
df.seriesTemporales <- func.toDataFrame(grid = grid, func = mean)
b = Sys.time()
b - a 

100.00% completado

Time difference of 4.03098 mins

Usando multiprocesamiento:

In [13]:
library(parallel)
library(transformeR)
grid = get(load("../1.Upscaling/upscaling_ba/ba_grid_5.Rdata"))

In [14]:
parallel_apply <- function(i) {
  inicio <- filas_df_params_mes_inicios[i]
  final <- filas_df_params_mes_finales[i]
  parametros_mes <- df_params[inicio:final, ]
  resultado_mes <- apply(parametros_mes, 1, func.applyDataFrame)
  return(resultado_mes)
}

In [15]:
func.applyDataFrame <- function (row, func = mean){
    subgrid <- subsetGrid(grid, season = row["Mes"], lonLim = row["X"], latLim  = row["Y"])
    serie <- func(subgrid$Data)
    return (serie)
}

In [16]:
func.iniciosFinales_mes<- function (grid){
    lon = grid$xyCoords$x
    lat = grid$xyCoords$y
    df_params <- expand.grid( "X" = lon, "Y" = lat, "Mes" = c(1:12))
    filas_df_params_mes <- (seq(0, nrow(df_params), length(lon) * length(lat)))
    
    filas_df_params_mes_finales = c()
    for (i in filas_df_params_mes[2:length(filas_df_params_mes)]){
        filas_df_params_mes_finales = c(filas_df_params_mes_finales, i)
    }
    filas_df_params_mes_inicios = c(1)
    for (i in seq(1,12)){
            lista_finales = filas_df_params_mes_finales[2:length(filas_df_params_mes_finales)-1]
            filas_df_params_mes_inicios = c(filas_df_params_mes_inicios, lista_finales[i]+1)    
    }
    filas_df_params_mes_inicios = na.omit(filas_df_params_mes_inicios)
    return (list(filas_df_params_mes_inicios,filas_df_params_mes_finales,df_params))
}


In [17]:
iniciosFinalesMes <- func.iniciosFinales_mes(grid)
filas_df_params_mes_inicios <- iniciosFinalesMes[[1]]
filas_df_params_mes_finales <- iniciosFinalesMes[[2]]
df_params <- iniciosFinalesMes[[3]]

In [18]:
func.toDataFrame <- function(grid, func = mean){
    lon = grid$xyCoords$x
    lat = grid$xyCoords$y
    df_params <- expand.grid( "X" = lon, "Y" = lat, "Mes" = c(1:12))
    df.seriesTemporales <-  expand.grid( "X" = lon, "Y" = lat)

    num_cores <- detectCores()
    cl <- makeCluster(num_cores)
    clusterExport(cl, c("filas_df_params_mes_inicios", "filas_df_params_mes_finales", "df_params", "func.applyDataFrame","subsetGrid","grid"))
    resultados <- parLapply(cl, 1:12, parallel_apply)

    stopCluster(cl)
    df.seriesTemporales <- cbind(df.seriesTemporales, resultados)
    colnames(df.seriesTemporales) <- c('X','Y', '1','2','3','4','5','6','7','8','9','10','11','12')
    df.seriesTemporales <- df.seriesTemporales[order(df.seriesTemporales$'X'),]
    df.seriesTemporales <- df.seriesTemporales[,3:14]
    return(df.seriesTemporales)
    
}

In [7]:
a = Sys.time()
df.seriesTemporales <- func.toDataFrame(grid = grid, func = mean)
b = Sys.time()
b - a 

Time difference of 38.55369 secs

In [115]:
func.applyDataFrame_FBA <- function (row){
    subgrid <- subsetGrid(grid_fba_sinTiempo, lonLim = row["X"], latLim  = row["Y"])
    serie <- subgrid$Data
    return (serie)
}

grid_fba <- get(load(ruta_grid_fba))
grid_fba_sinTiempo <- subsetGrid(grid_fba, year = 2020, season = 1)

func.toDataFrame_FBA <- function(grid_fba_sinTiempo){
    lon = grid_fba_sinTiempo$xyCoords$x
    lat = grid_fba_sinTiempo$xyCoords$y
    df_params_fba <- expand.grid( "X" = lon, "Y" = lat)
    resultados <- resultados <- apply(df_params_fba, 1, func.applyDataFrame_FBA)
    df.FBA <- cbind(df_params_fba, resultados)
    colnames(df.FBA) <- c('X','Y', 'FBA')
    df.FBA <- df.FBA[order(df.FBA$'X'),]
    return(df.FBA)
    
}

In [108]:
ruta_grid <- '../1.Upscaling/upscaling_fba/ fba_grid_5.Rdata'

grid_fba <- get(load(ruta_grid))
grid_fba_sinTiempo <- subsetGrid(grid_fba, year = 2020, season = 1)

In [116]:
grid_fba_sinTiempo <- subsetGrid(grid_fba, year = 2020, season = 1)
a = Sys.time()

df_fba_05 <- func.toDataFrame_FBA(grid_fba_sinTiempo)
b = Sys.time()
b - a 

Time difference of 23.03744 secs

In [117]:
lon = grid_fba$xyCoords$x
lat = grid_fba$xyCoords$y
df_params_fba <- expand.grid( "X" = lon, "Y" = lat)

,X,Y,FBA
,<dbl>,<dbl>,<dbl>
1,-177.5,-87.5,0.00000000
73,-177.5,-82.5,0.00000000
145,-177.5,-77.5,0.00000000
217,-177.5,-72.5,0.00000000
289,-177.5,-67.5,0.00000000
361,-177.5,-62.5,0.00000000
433,-177.5,-57.5,0.00000000
505,-177.5,-52.5,0.00000000
577,-177.5,-47.5,0.00000000
